#  Importing Important modules

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import tensorflow as tf
from tensorflow import keras

import os
import shutil
import random

import cv2
import albumentations as A
from PIL import Image

# Data Loading

* resize images to (224,224)
* split the training data to 80% train 20% validation

In [2]:
image_size = (224, 224)  # Resize to this size
batch_size = 32
test_split = 0.2
seed = 123
dataset_dir = '/kaggle/input/plant-village-augmented/dataset/content/Plant_leave_diseases_dataset_with_augmentation'

* using bilinear mode for resizing
* using label encoder for the classes


In [3]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    validation_split=test_split,  # Splitting dataset
    subset="training",            # Training subset
    seed=seed,
    image_size=image_size,        # Resizing images
    batch_size=batch_size,
    interpolation="bilinear",
    label_mode='int' 
)

Found 157521 files belonging to 39 classes.
Using 126017 files for training.


In [4]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    validation_split=test_split,  # Splitting dataset
    subset="validation",          # Validation subset
    seed=seed,
    image_size=image_size,        # Resizing images
    batch_size=batch_size,
    interpolation="bilinear",      # Interpolation method (you can use 'bicubic' here)
    label_mode='int'
)

Found 157521 files belonging to 39 classes.
Using 31504 files for validation.


In [5]:
test_dir = '/kaggle/input/plant-village-augmented/test/content/Test'
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=image_size,        # Resizing images
    batch_size=batch_size,
    interpolation="bilinear"      # Interpolation method (you can use 'bicubic' here)
    ,label_mode='int'
)

Found 39367 files belonging to 39 classes.


* optimize memory and cpu for training and loading data

In [6]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_ds.prefetch(buffer_size=AUTOTUNE)
validation_dataset = val_ds.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_ds.prefetch(buffer_size=AUTOTUNE)

# building model

* function to take pretrained model and modify last layer for the outputs


In [7]:
def finetune_model(model):
    model.trainable = False
    inputs = keras.Input(shape=(224,224,3))
    scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
    x = scale_layer(inputs)
    x = model(x, training=False)
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
    outputs = keras.layers.Dense(39,activation='softmax')(x)
    model = keras.Model(inputs, outputs)
    model.summary(show_trainable=True)
    return model

Loading the models to train on (Resnet50V2, MobileNetV2, VGG16)

In [8]:
resnet_model = tf.keras.applications.ResNet50V2(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False,
)

mobilenet_model = tf.keras.applications.MobileNetV2(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False,
)

Vggnet_model = tf.keras.applications.VGG16(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False,
)


94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


add models to ann array

In [9]:
models = [finetune_model(resnet_model),finetune_model(mobilenet_model),finetune_model(Vggnet_model)]

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ input_layer_3 (InputLayer)  │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ rescaling (Rescaling)       │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ resnet50v2 (Functional)     │ (None, 7, 7, 2048)    │ 23,564,800 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ global_average_pooling2d    │ (None, 2048)          │          0 │   -   │
│ (GlobalAveragePooling2D)    │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dropout (Dropout)           │ (None, 2048)          │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense (Dense)               │ (None, 39)            │     79,911 │   Y   │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 23,644,711 (90.20 MB)

 Trainable params: 79,911 (312.15 KB)

 Non-trainable params: 23,564,800 (89.89 MB)

...

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ input_layer_5 (InputLayer)  │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ rescaling_2 (Rescaling)     │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ vgg16 (Functional)          │ (None, 7, 7, 512)     │ 14,714,688 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ global_average_pooling2d_2  │ (None, 512)           │          0 │   -   │
│ (GlobalAveragePooling2D)    │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dropout_2 (Dropout)         │ (None, 512)           │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_2 (Dense)             │ (None, 39)            │     20,007 │   Y   │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 14,734,695 (56.21 MB)

 Trainable params: 20,007 (78.15 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

# Model Sampling Test
take a sample to train on to eliminate one of the models

In [10]:
subset_train_ds = train_ds.take(25250)

In [11]:
subset_val_ds = val_ds.take(5000)

In [12]:
model_names = ['Resnet50 V2','MobileNet V2', "VggNet 16"]

In [13]:
models_outputs = []
for i,model in enumerate(models):
    print("training for model: "+ model_names[i])
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    , optimizer = 'adam', metrics = ['accuracy'])
    model.fit(subset_train_ds, epochs=5,steps_per_epoch = 200, validation_data=subset_val_ds)
    models_outputs.append(model)

training for model: Resnet50 V2
Epoch 1/5


I0000 00:00:1725925316.898183      72 service.cc:145] XLA service 0x7cac5c0014d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1725925316.898233      72 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


  5/200 ━━━━━━━━━━━━━━━━━━━━ 7s 39ms/step - accuracy: 0.0053 - loss: 4.3826    

I0000 00:00:1725925323.480204      72 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


200/200 ━━━━━━━━━━━━━━━━━━━━ 64s 256ms/step - accuracy: 0.3261 - loss: 2.6048 - val_accuracy: 0.6857 - val_loss: 1.1111
Epoch 2/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 43s 216ms/step - accuracy: 0.6649 - loss: 1.1338 - val_accuracy: 0.7403 - val_loss: 0.8751
Epoch 3/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 43s 217ms/step - accuracy: 0.7041 - loss: 0.9764 - val_accuracy: 0.7591 - val_loss: 0.7895
Epoch 4/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 43s 217ms/step - accuracy: 0.7393 - loss: 0.8874 - val_accuracy: 0.7821 - val_loss: 0.7092
Epoch 5/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 43s 217ms/step - accuracy: 0.7574 - loss: 0.7840 - val_accuracy: 0.7941 - val_loss: 0.6767
training for model: MobileNet V2
Epoch 1/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 36s 135ms/step - accuracy: 0.2751 - loss: 2.8568 - val_accuracy: 0.6226 - val_loss: 1.3430
Epoch 2/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 22s 109ms/step - accuracy: 0.6038 - loss: 1.3583 - val_accuracy: 0.6765 - val_loss: 1.0924
Epoch 3/5
200/200 ━━━━━━━━━━━━━━━━━━━━ 22s 110ms/step - accuracy: 0

In [14]:
test_subset = test_ds.take(1000)
    

In [15]:
for model in models_outputs:
    model.evaluate(test_subset)

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 38s 38ms/step - accuracy: 0.7870 - loss: 0.6988
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 19s 19ms/step - accuracy: 0.7279 - loss: 0.8866
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 62s 62ms/step - accuracy: 0.5918 - loss: 1.8498


Vgg has the lowest performance so it will be Eliminated and i will continue with ResnetV250 and MobileNet with adjustments

# ResnetV2 50 Model

In [16]:
resnet_model = tf.keras.applications.ResNet50V2(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False,
)

In [17]:
resnet_model.trainable = False
inputs = keras.Input(shape=(224,224,3))
scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
x = scale_layer(inputs)
x = resnet_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(256,activation = 'relu')(x)
x = keras.layers.Dense(128,activation = 'relu')(x)
x = keras.layers.Dense(64, activation = 'relu')(x)
outputs = keras.layers.Dense(39,activation='softmax')(x)
resnet_model = keras.Model(inputs, outputs)
resnet_model.summary(show_trainable=True)

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ input_layer_7 (InputLayer)  │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ rescaling_3 (Rescaling)     │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ resnet50v2 (Functional)     │ (None, 7, 7, 2048)    │ 23,564,800 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ global_average_pooling2d_3  │ (None, 2048)          │          0 │   -   │
│ (GlobalAveragePooling2D)    │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_3 (Dense)             │ (None, 256)           │    524,544 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_4 (Dense)             │ (None, 128)           │     32,896 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_5 (Dense)             │ (None, 64)            │      8,256 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_6 (Dense)             │ (None, 39)            │      2,535 │   Y   │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 24,133,031 (92.06 MB)

 Trainable params: 568,231 (2.17 MB)

 Non-trainable params: 23,564,800 (89.89 MB)

In [18]:
es=keras.callbacks.EarlyStopping(monitor='val_accuracy',verbose=1,patience=7,mode='auto')
mc=keras.callbacks.ModelCheckpoint(filepath='/content/model.keras',monitor='val_accuracy',verbose=1,save_best_only=True)
lr=keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',verbose=1,patience=5,min_lr=0.001)

In [19]:
resnet_model.compile(optimizer='Adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])

In [20]:
resnet_model.fit(train_dataset,validation_data=validation_dataset,epochs=30,steps_per_epoch=200,verbose=1,callbacks=[mc,es,lr])

Epoch 1/30
199/200 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.3007 - loss: 2.5973
Epoch 1: val_accuracy improved from -inf to 0.65150, saving model to /content/model.keras
200/200 ━━━━━━━━━━━━━━━━━━━━ 58s 243ms/step - accuracy: 0.3024 - loss: 2.5904 - val_accuracy: 0.6515 - val_loss: 1.1417 - learning_rate: 0.0010
Epoch 2/30
199/200 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.6542 - loss: 1.1001
Epoch 2: val_accuracy improved from 0.65150 to 0.70220, saving model to /content/model.keras
200/200 ━━━━━━━━━━━━━━━━━━━━ 44s 222ms/step - accuracy: 0.6543 - loss: 1.0996 - val_accuracy: 0.7022 - val_loss: 0.9487 - learning_rate: 0.0010
Epoch 3/30
199/200 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.6923 - loss: 0.9737
Epoch 3: val_accuracy improved from 0.70220 to 0.73280, saving model to /content/model.keras
200/200 ━━━━━━━━━━━━━━━━━━━━ 44s 222ms/step - accuracy: 0.6924 - loss: 0.9733 - val_accuracy: 0.7328 - val_loss: 0.8351 - learning_rate: 0.0010
Epoch 4/30
199/200 ━━━━━━━━━━━━

/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 20: val_accuracy improved from 0.82793 to 0.82812, saving model to /content/model.keras
200/200 ━━━━━━━━━━━━━━━━━━━━ 45s 226ms/step - accuracy: 0.8267 - loss: 0.5202 - val_accuracy: 0.8281 - val_loss: 0.5201 - learning_rate: 0.0010
Epoch 21/30
199/200 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.8204 - loss: 0.5245
Epoch 21: val_accuracy improved from 0.82812 to 0.83970, saving model to /content/model.keras
200/200 ━━━━━━━━━━━━━━━━━━━━ 44s 222ms/step - accuracy: 0.8205 - loss: 0.5244 - val_accuracy: 0.8397 - val_loss: 0.4902 - learning_rate: 0.0010
Epoch 22/30
199/200 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.8357 - loss: 0.5054
Epoch 22: val_accuracy did not improve from 0.83970
200/200 ━━━━━━━━━━━━━━━━━━━━ 43s 217ms/step - accuracy: 0.8357 - loss: 0.5055 - val_accuracy: 0.8256 - val_loss: 0.5311 - learning_rate: 0.0010
Epoch 23/30
199/200 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.8253 - loss: 0.5096
Epoch 23: val_accuracy did not improve from 0.83970
200/200 ━━━

In [21]:
resnet_model.evaluate(test_dataset)

1231/1231 ━━━━━━━━━━━━━━━━━━━━ 51s 41ms/step - accuracy: 0.8442 - loss: 0.4781


[0.47578904032707214, 0.8450478911399841]

try with freeze 45 layers

In [22]:
resnet_model.save('resnet_frozen_layers.h5')

In [23]:
resnet_model = tf.keras.applications.ResNet50V2(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False,
)

In [24]:
len(resnet_model.layers)

190

In [25]:
for layer in resnet_model.layers[:150]:
    layer.trainable = False

In [26]:
inputs = keras.Input(shape=(224,224,3))
scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
x = scale_layer(inputs)
x = resnet_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(256,activation = 'relu')(x)
x = keras.layers.Dense(128,activation = 'relu')(x)
x = keras.layers.Dense(64, activation = 'relu')(x)
outputs = keras.layers.Dense(39,activation='softmax')(x)
resnet_model = keras.Model(inputs, outputs)
resnet_model.summary(show_trainable=True)

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ input_layer_9 (InputLayer)  │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ rescaling_4 (Rescaling)     │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ resnet50v2 (Functional)     │ (None, 7, 7, 2048)    │ 23,564,800 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ global_average_pooling2d_4  │ (None, 2048)          │          0 │   -   │
│ (GlobalAveragePooling2D)    │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_7 (Dense)             │ (None, 256)           │    524,544 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_8 (Dense)             │ (None, 128)           │     32,896 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_9 (Dense)             │ (None, 64)            │      8,256 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_10 (Dense)            │ (None, 39)            │      2,535 │   Y   │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 24,133,031 (92.06 MB)

 Trainable params: 15,802,279 (60.28 MB)

 Non-trainable params: 8,330,752 (31.78 MB)

In [27]:
resnet_model.compile(optimizer='Adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])

In [28]:
resnet_model.fit(train_dataset,validation_data=validation_dataset,epochs=30,steps_per_epoch=200,verbose=1,callbacks=[mc,es,lr])

Epoch 1/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.3734 - loss: 2.3836
Epoch 1: val_accuracy did not improve from 0.84732
200/200 ━━━━━━━━━━━━━━━━━━━━ 69s 259ms/step - accuracy: 0.3741 - loss: 2.3803 - val_accuracy: 0.5033 - val_loss: 6.6596 - learning_rate: 0.0010
Epoch 2/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.7206 - loss: 0.9112
Epoch 2: val_accuracy did not improve from 0.84732
200/200 ━━━━━━━━━━━━━━━━━━━━ 48s 241ms/step - accuracy: 0.7208 - loss: 0.9107 - val_accuracy: 0.6932 - val_loss: 5.0842 - learning_rate: 0.0010
Epoch 3/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.7715 - loss: 0.7211
Epoch 3: val_accuracy did not improve from 0.84732
200/200 ━━━━━━━━━━━━━━━━━━━━ 48s 240ms/step - accuracy: 0.7715 - loss: 0.7210 - val_accuracy: 0.7261 - val_loss: 1.7585 - learning_rate: 0.0010
Epoch 4/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.8096 - loss: 0.5997
Epoch 4: val_accuracy did not improve from 0.84732
200/200 ━━━━━━━━

In [29]:
resnet_model.evaluate(test_dataset)

1231/1231 ━━━━━━━━━━━━━━━━━━━━ 47s 39ms/step - accuracy: 0.9112 - loss: 0.2783


[0.2791484296321869, 0.9117789268493652]

In [30]:
resnet_model.save('resnet_frozen_150_layers.h5')